## _*Iene*_ - Boletim - Paridade e Cotação de moeda por data

- Retorna os boletins diários com a Paridade de Compra e a Paridade de Venda, a Cotação de Compra e a Cotação de Venda para a data da moeda consultada. São cinco boletins para cada data, um de abertura, três intermediários e um de fechamento.

*Parâmetros*

| Nome        | Tipo     | Título               | Descrição                                                                                                                                                                |
|-------------|----------|----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| moeda       | texto    | Moeda                | Código texto da moeda cujas cotações e paridades serão consultadas. Formato - MMM: Três letras. Exemplo: EUR. As moedas que estão disponíveis podem ser consultadas no recurso 'Moedas'. |
| dataCotacao | texto    | Data da cotação      | Data da cotação - informar no padrão 'MM-DD-AAAA'.                                                                                                                       |
| $format     | texto    | $format              | Tipo de conteúdo que será retornado.                                                                                                                                      |
| $select     | texto    | $select              | Propriedades que serão retornadas.                                                                                                                                        |
| $filter     | texto    | $filter              | Filtro de seleção de entidades. e.g. Nome eq 'João'. Clique aqui para as opções de operadores e funções.                                                                 |
| $orderby    | texto    | $orderby             | Propriedades para ordenação das entidades. e.g. Nome asc, Idade desc.                                                                                                     |
| $skip       | inteiro  | $skip                | Índice (maior ou igual a zero) da primeira entidade que será retornada.                                                                                                   |
| $top        | inteiro  | $top                 | Número máximo (maior que zero) de entidades que serão retornadas.                                                                                                         |


*Resultado*

| Nome             | Tipo    | Título                | Descrição                                                                                                                                                        |
|------------------|---------|------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| paridadeCompra   | decimal | Paridade de compra     | Paridade de compra da moeda consultada contra o dólar: Moedas tipo A: USD/[moeda]. Moedas tipo B: [moeda]/USD.                                                   |
| paridadeVenda    | decimal | Paridade de venda      | Paridade de venda da moeda consultada contra o dólar: Moedas tipo A: USD/[moeda]. Moedas tipo B: [moeda]/USD.                                                    |
| cotacaoCompra    | decimal | Cotação de compra      | Cotação de compra da moeda consultada contra a unidade monetária corrente: unidade monetária corrente/[moeda].                                                    |
| cotacaoVenda     | decimal | Cotação de venda       | Cotação de venda da moeda consultada contra a unidade monetária corrente: unidade monetária corrente/[moeda].                                                     |
| dataHoraCotacao  | texto   | Data e hora da cotação | Data, hora e minuto das paridades e cotações.                                                                                                                      |
| tipoBoletim      | texto   | Tipo do boletim        | Tipo das paridades e cotações para aquela data e hora. Podem ser dos tipos: Abertura, Intermediário, Fechamento Interbancário ou Fechamento.                     |


### 1. Importando bibliotecas necessárias

In [1]:
import pandas as pd  #possibilita manipulação e análise de dados
import requests #possibilita integração com serviços web, solicitações HTTP consumir dados de APIs
import json  #manipular arquivo json

### 2. Definindo a URL para a requisição da API

In [2]:
url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaDia(moeda=@moeda,dataCotacao=@dataCotacao)?@moeda='JPY'&@dataCotacao='03-06-2006'&$top=1000&$format=json&$select=paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim"

### 3. Criando a função para fazer a requisição e salvar o JSON na pasta Datasets

In [3]:
def requisicao_api(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:      #sucesso na requisição
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('datasets/jpyDia.json', 'w', encoding='utf-8') as arquivo:   #salvar arquivo json
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)

### 4. Executando a função para buscar os dados

In [4]:
requisicao_api (url)

Status Code: 200


### 5. Lendo o arquivo JSON salvo e transformando em DataFrame

In [5]:
jsondata = pd.read_json('datasets/jpyDia.json')

dados_df = pd.json_normalize(jsondata['value'])

### 6. Visualizando as primeiras linhas do DataFrame

In [6]:
dados_df.head()

,paridadeCompra,paridadeVenda,cotacaoCompra,cotacaoVenda,dataHoraCotacao,tipoBoletim
0,117.097,117.130,0.01804,0.01813,2006-03-06 09:19:00.0,Abertura
1,117.165,117.205,0.01810,0.01813,2006-03-06 09:30:00.0,Intermediário
2,117.221,117.311,0.01809,0.01812,2006-03-06 10:00:00.0,Intermediário
3,117.440,117.460,0.01806,0.01808,2006-03-06 10:30:00.0,Intermediário
4,117.460,117.480,0.01805,0.01806,2006-03-06 11:00:00.0,Intermediário


### 7. Função para salvar o DataFrame em CSV

In [7]:
def saveCsv(df: pd.DataFrame, nome_arquivo: str, separator: str, dec: str):
    df.to_csv(nome_arquivo, sep=separator, decimal=dec, index=False, encoding='utf-8-sig')


### 8. Definindo o caminho para salvar o CSV e salvando os dados

In [8]:
jpy_dia_csv = './datasets/jpy_dia_csv.csv'

saveCsv(dados_df, jpy_dia_csv, separator=';', dec=',')

print(f"DataFrame salvo com sucesso em: {jpy_dia_csv}")

DataFrame salvo com sucesso em: ./datasets/jpy_dia_csv.csv
